Parallel tree boosting. XGBOOST

In [1]:
"""from google.colab import drive
drive.mount('/content/gdrive')"""

"from google.colab import drive\ndrive.mount('/content/gdrive')"

In [2]:
"""%cd /content/gdrive/My Drive/Kaggle"""

'%cd /content/gdrive/My Drive/Kaggle'

In [3]:
"""import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"""

'import os\nos.environ[\'KAGGLE_CONFIG_DIR\'] = "/content/gdrive/My Drive/Kaggle'

In [4]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Nicolas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Nicolas\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Nicolas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Nicolas\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [5]:
import xgboost as xgb
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction import FeatureHasher
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.metrics import f1_score
from matplotlib import pyplot
from sklearn.preprocessing import LabelEncoder

ModuleNotFoundError: No module named 'xgboost'

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import datetime

In [ ]:
!kaggle competitions download -c nlp-getting-started

In [ ]:
!ls

In [ ]:
train=pd.read_csv("train.csv",encoding='latin-1')
datos =pd.read_csv("test.csv",encoding='latin-1')
datos.head()

In [ ]:
serie=datos['id']

In [ ]:
#Separo el target de los tweets
target = train.iloc[:,-1]

In [ ]:
#Aplico operaciones sobre train que luego replicare en el set de datos, para obtener los indices que yo considero relevantes
#y deshacerme de los que no lo son
#Empiezo por las arroba
train['cantidad@']=train['text'].str.count('@')
datos['cantidad@']=datos['text'].str.count('@')
train.head()

In [ ]:
#Aplico una funcion de hash sobre la location
#Primero fijo los NaN como la cadena "ninguna"
#Luego proceso eliminando mayusculas haciendo lemmatize
lemma=WordNetLemmatizer()
def reemplazarTexto(x):
  x=x.casefold()
  x=lemma.lemmatize(x)
  return x

def procesarLocation(df):
  df['location']=df['location'].fillna(value='none')
  return df['location'].apply(lambda x:reemplazarTexto(x))
train['location']=procesarLocation(train)
datos['location']=procesarLocation(datos)

In [ ]:
def crearFeatureHasher(serie,ndf):
  fh = FeatureHasher(n_features=6, input_type='string')
  sp = fh.fit_transform(serie)
  df = pd.DataFrame(sp.toarray(), columns=['fh1', 'fh2', 'fh3', 'fh4', 'fh5', 'fh6'])
  ndf=pd.concat([ndf, df], axis=1)
  return ndf
train=crearFeatureHasher(train['location'],train)
datos=crearFeatureHasher(datos['location'],datos)
train.head()

In [ ]:
#VERIFICAR QUE LOS LOCATION SEAN DE UBICACIONES VERDADERAS
paises=['afghanistan', 'albania', 'algeria', 'andorra', 'angola', 'antigua & barbuda', 'argentina', 'armenia', 'australia', 'austria','azerbaijan', 'bahamas', 'bahrain','bangladesh','barbados', 'belarus', 'belgium', 'belize', 'benin', 'bhutan', 'bolivia', 
       'bosnia & herzegovina', 'botswana', 'brazil', 'bulgaria', 'burkina faso', 'burundi', 'cabo verde', 'cambodia', 'cameroon', 
       'canada', 'chad', 'chile', 'china', 'colombia', 'comoros', 'congo', 'costa rica', 'croatia', 'cuba', 'cyprus', 'czech republic', 
       'denmark', 'djibouti', 'dominica', 'dominican republic', 'ecuador', 'egypt', 'el salvador', 'equatorial guinea', 'eritrea', 'estonia', 
       'eswatini', 'fiji', 'finland', 'france', 'gabon', 'gambia', 'georgia', 'germany', 'ghana', 'greece', 'grenada', 'guatemala', 
       'guinea', 'guinea-bissau', 'guyana', 'haiti', 'honduras', 'hungary', 'iceland', 'india', 'indonesia', 'iran', 
       'iraq', 'ireland', 'israel', 'italy', 'jamaica', 'japan', 'jordan', 'kazakhstan', 'kenya', 'kosovo', 'kuwait', 
       'kyrgyzstan', 'laos', 'latvia', 'lebanon', 'lesotho', 'liberia', 'libya', 'liechtenstein', 'lithuania', 'luxembourg', 
       'madagascar', 'malawi', 'malaysia kuala lumpur', 'maldives male', 'mali', 'malta', 'marshall', 'islands', 'mauritania', 'mauritius', 
       'mexico', 'micronesia', 'moldova', 'monaco', 'mongolia', 'montenegro', 'morocco', 'mozambique', 'myanmar', 'namibia', 'nepal', 
       'netherlands', 'new zealand', 'nicaragua', 'niger', 'nigeria', 'norway', 'pakistan', 
       'palestine', 'panama', 'papua new guinea', 'paraguay', 'peru', 'philippines', 'poland', 'portugal', 'qatar', 'romania', 
       'russia moscow', 'rwanda kigali', 'saudi arabia', 'serbia', 'seychelles', 'sierra leone', 'singapore', 'slovakia', 'slovenia','south africa', 'south korea', 
       'south sudan juba', 'spain', 'sri lanka', 'sudan', 'suriname', 'sweden ', 'switzerland bern', 'syria', 'tajikistan','thailand', 'turkey', 'turkmenistan', 'uganda', 'ukraine', 'united arab emirates', 'uae',
       'united kingdom', 'uk', 'united states of america', 'usa', 'uruguay', 'uzbekistan', 
       'vatican city', 'holy see', 'venezuela',  'vietnam', 'yemen', 'zambia','zimbabwe']
capitales=['kabul','tirana','algiers','andorra la vella','luanda','buenos aires','yerevan','canberra','vienna','baku','manama','dhaka','minsk','bridgetown','brussels',
           'belmopan','porto-novo','thimphu','sucre','sarajevo','gaborone','brasilia','sofia','bujumbura','praia','phnom penh','yaounde','ottawa','santiago','beijing','bogotá',' moroni','san jose',
           ' zagreb','havana','nicosia','prague','copenhagen','roseau','santo domingo','quito','cairo','san salvador','malabo','asmara','tallinn','mbabane','suva','helsinki','paris','libreville','tbilisi','berlin',
           'accra','athens','guatemala city','mexico city','mexico df','conakry','bissau','georgetown','port-au-prince','tegucigalpa','budapest','reykjavik','new delhi','jakarta','tehran','baghdad','dublin','jerusalem',
           'rome','kingston','tokyo','amman','nairobi','pristina','kuwait city','bishkek','vientiane','riga','beirut','maseru','monrovia','tripoli','vaduz','vilnius','luxembourg','antananarivo','lilongwe',
           'monaco','podgorica','bamako','valletta','majuro','nouakchott','windhoek','rabat','maputo','kathmandu','port louis','amsterdam','wellington','managua','niamey','abuja','colombo','belgrade',
           'panama city','east jerusalem','port moresby','asunción','lima','madrid','khartoum','paramaribo','stockholm','freetown','singapore','caracas','damascus','dushanbe','harare','bangkok',
           'ankara','bucharest','riyadh','manila','warsaw','oslo','islamabad','lisbon','victoria','bratislava','seoul','kiev','london','abu dhabi','washington','washington dc','washington d.c.', 'd.c.','montevideo','tashkent', 'vatican city',
           'hanoi']
estadosUS= ["alabama","alaska","arizona","arkansas","california","colorado",\
  "connecticut","delaware","florida","georgia","hawaii","idaho","illinois",\
  "indiana","iowa","kansas","kentucky","louisiana","maine","maryland",\
  "massachusetts","michigan","minnesota","mississippi","missouri","montana",\
  "nebraska","nevada","new hampshire","new jersey","new mexico","new york",\
  "north carolina","north dakota","ohio","oklahoma","oregon","pennsylvania",\
  "rhode island","south carolina","south dakota","tennessee","texas","utah",\
  "vermont","virginia","washington","west virginia","wisconsin","wyoming"]
train['locationPais']=train['location'].str.contains('|'.join(paises))*1
datos['locationPais']=datos['location'].str.contains('|'.join(paises))*1
train['locationCiudad']=(train['location'].str.contains('|'.join(capitales)) | train['location'].str.contains('|'.join(estadosUS)))*1
datos['locationCiudad']=(datos['location'].str.contains('|'.join(capitales)) | datos['location'].str.contains('|'.join(estadosUS)))*1
train.head()


In [ ]:
#Analizo longitud de los tweets
#Convertir lo categorico en binario usando one hot encoding
train['longitudTweet']=train.text.str.len()
datos['longitudTweet']=datos.text.str.len()

bins =[-np.inf,20, 40, 60,80, 100, 120, 140, 160, 180, 200, np.inf]
l=['0-20l','20-40l','40-60l','60-80l','80-100l','100-120l','120-140l','140-160l','160-180l','180-200l','200+l']
trainCatLong=pd.cut(train['longitudTweet'], bins,labels=l)
datosCatLong=pd.cut(datos['longitudTweet'], bins,labels=l)

In [ ]:
#One hot encoding
train = pd.concat([train,pd.get_dummies(trainCatLong)],axis=1)
datos = pd.concat([datos,pd.get_dummies(datosCatLong)],axis=1)
datos.head(1)

In [ ]:
#Analizo la cantidad de Hashtags en un tweet
train['cantidad#']=train['text'].str.count('#')
datos['cantidad#']=datos['text'].str.count('#')

In [ ]:
#Hago binary encoding de la cantidad de hashtags en un tweet
import math
def convertBin(x,maximo):
  lista=[]
  binario=format(x,'02b')
  while len(binario)<maximo:
    binario='0'+binario
  for i in binario:
    lista.append(int(i))
  return lista

def binaryEncoding(serie,longMax):
  longMax+=1
  binaryHash=serie.apply(lambda x:convertBin(x,longMax))
  ncol=[]
  for i in range(longMax-1,-1,-1):
    ncol.append(str(i)+'b'+serie.name)
  return pd.DataFrame((item for item in binaryHash),columns=ncol)

In [ ]:
longMax=max(math.floor(math.log(train['cantidad#'].max(),2)),math.floor(math.log(datos['cantidad#'].max(),2)))
#Calculo asi long max porque el train y test tienen que tener la misma cant de col
train = pd.concat([train,binaryEncoding(train['cantidad#'],longMax)],axis=1)
datos = pd.concat([datos,binaryEncoding(datos['cantidad#'],longMax)],axis=1)

In [ ]:
#Hago binary encoding de la cantidad de @ en los tweets
longMax=max(math.floor(math.log(train['cantidad@'].max(),2)),math.floor(math.log(datos['cantidad@'].max(),2)))
#Calculo asi long max porque el train y test tienen que tener la misma cant de col
train = pd.concat([train,binaryEncoding(train['cantidad@'],longMax)],axis=1)
datos = pd.concat([datos,binaryEncoding(datos['cantidad@'],longMax)],axis=1)
train.head(1)

In [ ]:
#Cuento si el texto contiene un http y cuantos tiene
def analizarHttp(x):
  tokens=nltk.word_tokenize(x)
  resultado=0
  for token in tokens:
    resultado += (token=='http')*1      
  return resultado

train['cantidadLinks']=train['text'].apply(lambda x:analizarHttp(x))
datos['cantidadLinks']=datos['text'].apply(lambda x:analizarHttp(x))

In [ ]:
#Hago binary encoding de https
longMax=max(math.floor(math.log(train['cantidadLinks'].max(),2)),math.floor(math.log(datos['cantidadLinks'].max(),2)))
#Calculo asi long max porque el train y test tienen que tener la misma cant de col
train = pd.concat([train,binaryEncoding(train['cantidadLinks'],longMax)],axis=1)
datos = pd.concat([datos,binaryEncoding(datos['cantidadLinks'],longMax)],axis=1)
train.head(1)

In [ ]:
import re
def reemplazarTexto(x):
  x=x.casefold()
  x=lemma.lemmatize(x)
  x=x.casefold()
  x=re.sub(r'http\S*', 'http:',x)
  x=re.sub(r'@[A-Za-z0-9]+','',x)
  x=re.sub(r'[^a-z\s]', '',x)
  return x
train['textoProcesado']=train['text'].apply(lambda x:reemplazarTexto(x))
datos['textoProcesado']=datos['text'].apply(lambda x:reemplazarTexto(x))

In [ ]:
#Primero preproceso eliminando mayusculas haciendo lemmatize
train['keyword'].fillna(value='ninguna',inplace=True)
datos['keyword'].fillna(value='ninguna',inplace=True)
lemma=WordNetLemmatizer()
train['keyword']=train['keyword'].apply(lambda x:reemplazarTexto(x))
datos['keyword']=datos['keyword'].apply(lambda x:reemplazarTexto(x))

In [ ]:
#Analizo si los hashtags contienen alguna keyword
def analizarHashtag(x,keys):
  tokens=nltk.word_tokenize(x)
  resultado=False
  for i in range(len(tokens)):
    if ((tokens[i]=='#') & (i!=len(tokens)-1)):
      resultado = lemma.lemmatize(tokens[i+1]) in keys
  return resultado*1
lTrainK=train['keyword'].unique().tolist()
lDatosK=train['keyword'].unique().tolist()
train['hashtagKey'] = train['textoProcesado'].apply(lambda x:analizarHashtag(x,lTrainK))
datos['hashtagKey'] = datos['textoProcesado'].apply(lambda x:analizarHashtag(x,lDatosK))

In [ ]:
#Analizo si se menciona algun tipo de entretenimiento en el tweet
#Puede ser juegos, musica, peliculas...
listaEntretenimiento=['music','radio','tv','show','dj','game','gaming','videogame','history','movies','youtube','book','ebook','sports','esports']

def analizarContenido(x):
  tokens=nltk.word_tokenize(x)
  resultado=0
  for palabra in tokens:
    if palabra in listaEntretenimiento:
      resultado = 1
  return resultado

train['entretenimiento'] = train['textoProcesado'].apply(lambda x:analizarContenido(x))
datos['entretenimiento'] = datos['textoProcesado'].apply(lambda x:analizarContenido(x))

In [ ]:
#Analizo cantidad de palabras en los tweets
def contarPalabras (d):
  return (len(str.split(d,' ')))
train['cantidadPalabras']=train['text'].apply(lambda x: contarPalabras(x))
datos['cantidadPalabras']=datos['text'].apply(lambda x: contarPalabras (x))

In [ ]:
#Hago hashing trick para keywords
def crearFeatureHasher(serie,ndf):
  fh = FeatureHasher(n_features=6, input_type='string')
  sp = fh.fit_transform(serie)
  df = pd.DataFrame(sp.toarray(), columns=['kh1', 'kh2', 'kh3', 'kh4', 'kh5', 'kh6'])
  ndf=pd.concat([ndf, df], axis=1)
  return ndf
train=crearFeatureHasher(train['keyword'],train)
datos=crearFeatureHasher(datos['keyword'],datos)
train.head(1)

In [ ]:
#Analizo la relacion entre cantidad de # y la longitud del tweet
#Hacer esto categorico
train['relacion#longitud']=train['cantidad#']/train['cantidadPalabras']*100
datos['relacion#longitud']=datos['cantidad#']/datos['cantidadPalabras']*100
l=['pocos#','medio#','muchos#']
trainCatLong=pd.cut(train['relacion#longitud'], 3,labels=l)
datosCatLong=pd.cut(datos['relacion#longitud'], 3,labels=l)
#One hot encoding
train = pd.concat([train,pd.get_dummies(trainCatLong)],axis=1)
datos = pd.concat([datos,pd.get_dummies(datosCatLong)],axis=1)

In [ ]:
#Analizo la relacion entre cantidad de @ y la longitud del tweet
#Hacer esto categorico
train['relacion@longitud']=train['cantidad@']/train['cantidadPalabras']*100
datos['relacion@longitud']=datos['cantidad@']/datos['cantidadPalabras']*100
l=['pocos@','medio@','muchos@']
trainCatLong=pd.cut(train['relacion@longitud'], 3,duplicates='drop',labels=l)
datosCatLong=pd.cut(datos['relacion@longitud'], 3,duplicates='drop',labels=l)
#One hot encoding
train = pd.concat([train,pd.get_dummies(trainCatLong)],axis=1)
datos = pd.concat([datos,pd.get_dummies(datosCatLong)],axis=1)
datos.head(1)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit(pd.concat([train,datos])['textoProcesado'].values.tolist())
matTrain=vectorizer.transform(train['textoProcesado'].values.tolist())
matDatos=vectorizer.transform(datos['textoProcesado'].values.tolist())

In [ ]:
train=pd.concat([train,pd.DataFrame(matTrain.todense(),columns=vectorizer.get_feature_names())],axis=1)
datos=pd.concat([datos,pd.DataFrame(matDatos.todense(),columns=vectorizer.get_feature_names())],axis=1)

In [ ]:
#Calculo la cantidad de stopwords por texto
from nltk.corpus import stopwords
def longitudSinStopword(x):
  tokens=nltk.word_tokenize(x)
  filtered_words = [word for word in tokens if word in stopwords.words('english')]
  return len(filtered_words)
datos['stopWordsXTexto']=pd.qcut(datos['textoProcesado'].apply(lambda x: longitudSinStopword(x))/datos['textoProcesado'].str.len(),q=5)
train['stopWordsXTexto']=pd.qcut((train['textoProcesado'].apply(lambda x: longitudSinStopword(x)))/train['textoProcesado'].str.len(),q=5)

In [ ]:
#Remuevo cosas que no hacen al analisis
entrenar=train.drop(columns=['target','id','keyword','location','text','textoProcesado','stopWordsXTexto','relacion#longitud','relacion@longitud','longitudTweet','cantidad@','cantidad#','cantidadPalabras'])
test=datos.drop(columns=['target','id','keyword','location','text','textoProcesado','stopWordsXTexto','relacion#longitud','relacion@longitud','longitudTweet','cantidad@','cantidad#','cantidadPalabras'])

In [ ]:
entrenar.head()

In [ ]:
#from sklearn.model_selection import train_test_split
#entrenar.info()
#X_train, X_test, y_train, y_test =  train_test_split(entrenar, target, test_size=0.2, random_state=123)

In [ ]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', 
                colsample_bytree = 0.4, learning_rate = 0.2,
                max_depth = 6, alpha = 10, n_estimators = 100)

In [ ]:
# split data into X and y
X = entrenar
Y = target
# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=7)
# fit model no training data
model = xg_reg
eval_set = [(X_test, y_test)]
model.fit(X_train, y_train, eval_metric="error")
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: %f" % (rmse))

In [ ]:
#Nota target es una columna en test por el TF idf
preds = xg_reg.predict(test)
preds = (preds>=0.5)*1
predictions = pd.DataFrame()
predictions['id']=serie
predictions['target']=preds
predictions.head()
predictions.to_csv('resultadosTFIDF.csv',index=False)

In [ ]:
#!kaggle competitions submit -c nlp-getting-started -f resultados.csv -m "Nuevo intento"